In [ ]:
from __future__ import division

import numpy as np

from numpy.linalg import svd, qr, cholesky

# Solves Ax = b with svd.
def solve_svd(A, b):
    assert len(A[0]) == len(b)
    
    U, S, V = svd(A)
    UTb = np.dot(np.transpose(U), b)
    w = np.divide(UTb, S)

    return np.dot(np.transpose(V), w)

def solve_QR(A, b):
    n = len(b)
    Q, R = qr(A)
    
    QTb = np.dot(np.transpose(Q), b)

    x = np.zeros(n)
    for i in xrange(n-1, -1, -1):
        x[i] = (QTb[i] - np.dot(R[i,i+1:n], x[i+1:n]))/R[i,i]

    return x

def solve_normal(A, b):
    n = len(b)
    L = cholesky(np.dot(np.transpose(A),A))
    LT = np.transpose(L)

    ATb = np.dot(np.transpose(A), b)
        
    w = np.zeros(n)
    for i in xrange(0, n):
        w[i] = (ATb[i] - np.dot(L[i,0:i], w[0:i]))/L[i,i]
        
    x = np.zeros(n)
    for i in xrange(n-1, -1, -1):
        x[i] = (w[i] - np.dot(LT[i,i+1:n], x[i+1:n]))/LT[i,i]

    return x

In [ ]:
from __future__ import division

from solve_algorithms import solve_svd, solve_QR, solve_normal

import numpy as np

# I could not find how to add seasonal terms
# and tried to add x^1/2 like terms but it turns out
# predictions are quite bad.

def weirdfunc(x, coef):
    a = coef
    
    ret = 0
    for i in xrange(len(a)):
        ret += a[i]*x**((-1)**i*(len(a)-i-1)/2.0)

    return ret

def weirdfit(x, y, dy= None, method= 'svd'):
    k = len(x)

    W = np.zeros((k,k))
    J = np.zeros((k,3))
    for i in xrange(3):
        J[:,i] = np.power(x, (-1)**i*i/2.0)

    np.fill_diagonal(W, np.power(dy,-2))

    JT = np.transpose(J)
    if method == 'svd':
        P = solve_svd(np.dot(JT, np.dot(W,J)),
                      np.dot(JT, np.dot(W,y)))
    if method == 'QR':
        P = solve_QR(np.dot(JT, np.dot(W,J)),
                     np.dot(JT, np.dot(W,y)))
    if method == 'normal':
        P = solve_normal(np.dot(JT, np.dot(W,J)),
                         np.dot(JT, np.dot(W,y)))

        
    return np.flipud(P)

In [ ]:
from __future__ import division, print_function

from weirdfitfunction import weirdfit, weirdfunc

from solve_algorithms import solve_svd, solve_QR, solve_normal

import numpy as np

import matplotlib.pyplot as plt

def prediction(data, test = []):
    t, l = data.shape
    clrs = ['red', 'blue', 'green', 'yellow', 'cyan', 'black'] 
    
    x = np.linspace(1, l, l)
    x_pre = np.linspace(l+1, l+6, 6)
    
    
    fig = plt.figure()
    for i in xrange(t):
        plt.plot(x, data[i], marker = 'o', label = 'item '+str(i),
                 color= clrs[i])
    plt.ylim(np.min(data)*0.9, np.max(data)*1.1)
    plt.xlim(0,l+7)
    plt.legend(loc= 'upper left', fontsize = 12)
    plt.savefig('fig1.eps')
    err = []
    for i in xrange(t):
        coeff_svd = weirdfit(x, data[i], dy = np.ones(l), method = 'svd')
        coeff_QR = weirdfit(x, data[i], dy = np.ones(l), method = 'QR')
        coeff_normal = weirdfit(x, data[i], dy = np.ones(l), method = 'normal')

        yfitted_svd = [weirdfunc(x0, coeff_svd) for x0 in x_pre]
        yfitted_QR = [weirdfunc(x0, coeff_QR) for x0 in x_pre]
        yfitted_normal = [weirdfunc(x0, coeff_normal) for x0 in x_pre]
        plt.plot(x_pre, yfitted_svd, color= clrs[i], marker= 's', ls= '')

        if test != []:
            err.append(np.linalg.norm(yfitted_svd-test[i]))
        
    plt.ylim(np.min(data)*0.9, np.max(data)*1.1)
    plt.xlim(0,l+7)
    plt.savefig('fig2.eps')

    #About relative errors
    ys = [yfitted_svd, yfitted_QR, yfitted_normal]
    print('Total relative error between predictions generated by different computational methods')
    print('SVD : SVD Decomposition')
    print('QR : QR decomposition')
    print('NormE: Normal Equations')
    print('\t SVD\t QR\t NormE')
    for y1 in ys:
        if y1 == yfitted_svd:
            print('SVD\t', end = '')
        if y1 == yfitted_QR:
            print('QR\t', end = '')
        if y1 == yfitted_normal:
            print('NormE\t', end = '')
        for y2 in ys:
            y1 = np.array(y1)
            y2 = np.array(y2)
            print(np.linalg.norm(y2-y1)/np.linalg.norm(y1), end = '\t')
        print()

    if test != []:
        print()
        print('Errors (Euclidian norms)')
        for i in xrange(t):
            print('Item ' + str(i) + ': ' + str(err[i]))
        print('Total : ' + str(np.sum(err)))

In [ ]:
from __future__ import division

from solve_algorithms import solve_svd, solve_QR, solve_normal

import numpy as np

import matplotlib.pyplot as plt

def distance(x, y):
    n = len(x)

    x = np.array(x)
    y = np.array(y)
    d = np.sum(np.power(x-y,2))

    return np.sqrt(d)

def rey_quo(A, x):
    return np.dot(np.transpose(x), np.dot(A, x))/np.linalg.norm(x)

def poweriter(A, w):
    n = len(A[0])
    b1 = np.ones(n)/np.sqrt(n)
    b2 = np.ones(n)
    c = 1
    o = 0
    while o < 1000:
        b2 = solve_normal(A, b1/c)
        c = np.linalg.norm(b2)
        b1 = b2
        o +=1
        
    return b2
        
# k-nearest neighbor graph
def knng(X, k= 5, sig= 0.3, tau= 0):
    n = len(X[0])
    m = len(X[:])

    if n == 2:
        fig = plt.figure()
        plt.plot(X[:,0], X[:,1], ls= '', marker = 'o', ms=3.5, color= 'blue')
        plt.savefig('fig1.eps')
        
    dist = np.zeros((m,m))
    dist2 = np.zeros((m,m))
    for i in xrange(m):
        for j in xrange(m):
            if i == j:
                dist[i][j] = 1000000000
                dist2[i][j] = 0
            else:
                dist2[i][j] = distance(X[i], X[j])
                dist[i][j] = distance(X[i], X[j])

    edgelist = []
    for i in xrange(m):
        edge = []
        for j in xrange(k):
            edge.append(np.argmin(dist[i,:]))
            dist[i, edge[j]] = 1000000000
        edgelist.append(edge)

    if n == 2:
        for i in xrange(m):
            for j in xrange(k):
                plt.plot([X[i][0], X[edgelist[i][j]][0]],
                         [X[i][1], X[edgelist[i][j]][1]], color= 'blue')

        plt.savefig('fig2.eps')

    W = np.zeros((m,m))
    D = np.zeros((m,m)) # actually D^-1
    for i in xrange(m):
        for j in xrange(k):
            W[i][edgelist[i][j]] = np.exp(-0.5*(dist2[i][edgelist[i][j]]/sig)**2)
        D[i][i] = 1/np.sum(W[i])
            
    # Find eigenvector
    L = np.identity(m) - np.dot(W, D)
    e_vec = poweriter(L, 0)
    e_val = rey_quo(L, e_vec)


    Labels = []
    X_clus1 = []
    for i in xrange(m):
        if e_vec[i] < tau:
            Labels.append(1)
            X_clus1.append(X[i])
        else:
            Labels.append(2)

    X_clus1 = np.array(X_clus1)
    plt.plot(X_clus1[:,0], X_clus1[:,1], ls= '', marker = 's',
             ms=4.5, color= 'red')
    plt.savefig('fig3.eps')
    
    # Histogram
    hist, bin_edges = np.histogram(e_vec)
    fig2 = plt.figure()
    plt.bar(bin_edges[0:-1], hist, bin_edges[1]-bin_edges[0])
    plt.savefig('fig4.eps')

    return Labels, W 
    
if __name__ == '__main__':
    np.random.seed(1992)
    
    m = 100
    n = 2
    X = []
    for i in xrange(m):
        X.append(np.random.rand(n))

    X = np.array(X)
    knng(X, k=5)
